In [52]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [53]:
books = pd.read_csv(
    "books.csv",
    sep=';',
    encoding='utf-8',
    encoding_errors='ignore',
    on_bad_lines='skip',
    low_memory=False 
)

ratings = pd.read_csv(
    "ratings.csv",
    sep=';',
    encoding='utf-8',
    encoding_errors='ignore',
    on_bad_lines='skip',
    low_memory=False 
)

users = pd.read_csv(
    "users.csv",
    sep=';',
    encoding='utf-8',
    encoding_errors='ignore',
    on_bad_lines='skip',
    low_memory=False 
)

In [ ]:
users.head()

In [4]:
books.head()

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...
2,0060973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...
4,0393045218,The Mummies of Urumchi,E. J. W. Barber,1999,W. W. Norton &amp; Company,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...


In [5]:
rating.head()

NameError: name 'rating' is not defined

In [ ]:
print(books.shape)
print(ratings.shape)
print(users.shape)

In [ ]:
ratings['User-ID'].value_counts()

In [ ]:
x = ratings['User-ID'].value_counts() > 100

In [ ]:
y = x[x].index.tolist()

In [ ]:
ratings = ratings[ratings['User-ID'].isin(y)]

In [ ]:
ratings.head()

In [ ]:
ratings_with_books = ratings.merge(books, on = "ISBN")

In [ ]:
ratings_with_books.head()

In [ ]:
books_rating_count = ratings_with_books.groupby('Book-Title')['Book-Rating'].count().reset_index()

In [ ]:
books_rating_count.head()

In [ ]:
books_rating_count.rename(columns={'Book-Rating' : 'num_of_rating'}, inplace = True)

In [ ]:
books_rating_count.head()

In [ ]:
ratings_with_books = ratings_with_books.merge(books_rating_count, on = "Book-Title")

In [ ]:
valued_books = ratings_with_books[ratings_with_books['num_of_rating'] > 50]

In [ ]:
valued_books.shape

In [ ]:
valued_books.drop_duplicates(['User-ID','Book-Title'], inplace= True)

In [ ]:
valued_books.head()

In [ ]:
book_pivot = valued_books.pivot_table(columns='User-ID', index='Book-Title', values = 'Book-Rating')

In [ ]:
book_pivot

In [ ]:
book_pivot.shape

In [ ]:
book_pivot.fillna(0, inplace = True)

In [ ]:
book_pivot

In [ ]:
from scipy.sparse import csr_matrix

In [ ]:
book_sparse = csr_matrix(book_pivot)

In [ ]:
from sklearn.neighbors import NearestNeighbors
model = NearestNeighbors(algorithm='brute')

In [ ]:
model.fit(book_sparse)

In [ ]:
result = book_pivot.index.get_loc('Harry Potter and the Chamber of Secrets (Book 2)')
print(result)

In [ ]:
distance, suggestion = model.kneighbors(book_pivot.iloc[329,:].values.reshape(1, -1), n_neighbors = 10)

In [ ]:
suggestion

In [ ]:
distance

In [ ]:
for i in range(len(suggestion)):
    print(book_pivot.index[suggestion[i]])

In [ ]:
book_names = book_pivot.index

In [ ]:
book_names

In [ ]:
import pickle
pickle.dump(model, open('artifacts/model.pkl', 'wb'))
pickle.dump(book_names, open('artifacts/book_names.pkl', 'wb'))
pickle.dump(valued_books, open('artifacts/final_ratings.pkl', 'wb'))
pickle.dump(book_pivot, open('artifacts/book_pivot.pkl', 'wb'))


In [ ]:
def recommending_books(book_name):
    book_id = np.where(book_pivot.index == book_name)[0][0]
    distance, suggestion = model.kneighbors(book_pivot.iloc[book_id,:].values.reshape(1, -1), n_neighbors = 10)
    for i in range(len(suggestion)):
        books = book_pivot.index[suggestion[i]]
        for j in books:
            print(j)
    

In [ ]:
recommending_books('Harry Potter and the Chamber of Secrets (Book 2)')

In [51]:
final_ratings

NameError: name 'final_ratings' is not defined